In [160]:
import pandas as pd

diagnosis = pd.read_csv('./Patient_Diagnosis.csv')
treatment = pd.read_csv('./Patient_Treatment.csv')
icd9cm_code = pd.read_excel('./CMS32_DESC_LONG_SHORT_DX.xlsx')
icd9cm_code.columns = ['Diagnosis_Code', 'Long_Description', 'Short_Description']
diagnosis['Diagnosis_Code'] *= 10
diagnosis['Diagnosis_Code'] = diagnosis['Diagnosis_Code'].astype(str).str.replace('.0', '')
treatment_diagnosis_df = pd.merge(treatment, diagnosis,  how='inner', left_on=['Patient_ID'], right_on = ['Patient_ID'])
tdc = pd.merge(treatment_diagnosis_df, icd9cm_code, how='inner', left_on=['Diagnosis_Code'], right_on=['Diagnosis_Code'])

In [246]:
# Diseases treated at the clinic
"""
Drop duplicates to find unique diseases treated at the clinic
"""
diseases = tdc.drop_duplicates(subset=['Diagnosis_Code','Diagnosis','Long_Description','Short_Description'])
diseases[['Diagnosis_Code','Diagnosis','Long_Description']]

,Diagnosis_Code,Diagnosis,Long_Description
0,1749,Breast Cancer,"Malignant neoplasm of breast (female), unspeci..."
238,1741,Breast Cancer,Malignant neoplasm of central portion of femal...
328,1539,Colon Cancer,"Malignant neoplasm of colon, unspecified site"
378,1533,Colon Cancer,Malignant neoplasm of sigmoid colon
453,1535,Colon Cancer,Malignant neoplasm of appendix vermiformis
548,1534,Colon Cancer,Malignant neoplasm of cecum
626,1745,Breast Cancer,Malignant neoplasm of lower-outer quadrant of ...
671,1748,Breast Cancer,Malignant neoplasm of other specified sites of...
805,1746,Breast Cancer,Malignant neoplasm of axillary tail of female ...
846,1743,Breast Cancer,Malignant neoplasm of lower-inner quadrant of ...


In [220]:
# How many patients does the clinic have for each cancer type?
"""
Group by diagnosis code, and find counts for each cancer type by being unique on Patient_ID
"""
code_counts = tdc.groupby('Diagnosis_Code')
code_counts = code_counts['Patient_ID'].nunique().to_frame("Patient_Counts").reset_index()
cancer_type_counts = pd.merge(code_counts, icd9cm_code, how='inner', left_on=['Diagnosis_Code'], right_on=['Diagnosis_Code'])
cancer_type_counts


,Diagnosis_Code,Patient_Counts,Long_Description,Short_Description
0,1533,2,Malignant neoplasm of sigmoid colon,Mal neo sigmoid colon
1,1534,4,Malignant neoplasm of cecum,Malignant neoplasm cecum
2,1535,3,Malignant neoplasm of appendix vermiformis,Malignant neo appendix
3,1536,1,Malignant neoplasm of ascending colon,Malig neo ascend colon
4,1537,1,Malignant neoplasm of splenic flexure,Mal neo splenic flexure
5,1538,1,Malignant neoplasm of other specified sites of...,Malignant neo colon NEC
6,1539,2,"Malignant neoplasm of colon, unspecified site",Malignant neo colon NOS
7,1741,4,Malignant neoplasm of central portion of femal...,Mal neo breast-central
8,1742,1,Malignant neoplasm of upper-inner quadrant of ...,Mal neo breast up-inner
9,1743,3,Malignant neoplasm of lower-inner quadrant of ...,Mal neo breast low-inner


In [225]:
# How long after being diagnosed do patients start treatment?
"""
Sort by treatment dates, and find the earliest one for each patient and diagnosis date.
Convert the date columns into datetime type.
Subtract Treatment_Date column with Diagnosis_Date column.
Store in a column.
"""
time = tdc.sort_values('Treatment_Date').groupby(['Patient_ID','Diagnosis_Date'], as_index=False).first()
time['Diagnosis_Date'] = pd.to_datetime(time['Diagnosis_Date'])
time['Treatment_Date'] = pd.to_datetime(time['Treatment_Date'])
time['Days_Until_Treatment'] = time['Treatment_Date'] - time['Diagnosis_Date']
time

,Patient_ID,Diagnosis_Date,Treatment_Date,Drug_Code,Diagnosis_Code,Diagnosis,Long_Description,Short_Description,Days
0,2038,2010-01-21,2010-01-24,A,1749,Breast Cancer,"Malignant neoplasm of breast (female), unspeci...",Malign neopl breast NOS,3 days
1,2120,2010-01-09,2010-01-24,A,1741,Breast Cancer,Malignant neoplasm of central portion of femal...,Mal neo breast-central,15 days
2,2407,2010-06-13,2010-06-19,A,1749,Breast Cancer,"Malignant neoplasm of breast (female), unspeci...",Malign neopl breast NOS,6 days
3,2425,2010-12-15,2011-01-12,B,1749,Breast Cancer,"Malignant neoplasm of breast (female), unspeci...",Malign neopl breast NOS,28 days
4,2462,2011-01-07,2011-01-11,A,1749,Breast Cancer,"Malignant neoplasm of breast (female), unspeci...",Malign neopl breast NOS,4 days
5,2634,2011-02-19,2011-02-23,B,1539,Colon Cancer,"Malignant neoplasm of colon, unspecified site",Malignant neo colon NOS,4 days
6,2735,2011-04-18,2011-04-23,B,1749,Breast Cancer,"Malignant neoplasm of breast (female), unspeci...",Malign neopl breast NOS,5 days
7,2763,2011-04-19,2011-04-23,A,1741,Breast Cancer,Malignant neoplasm of central portion of femal...,Mal neo breast-central,4 days
8,2770,2011-04-16,2011-04-22,B,1539,Colon Cancer,"Malignant neoplasm of colon, unspecified site",Malignant neo colon NOS,6 days
9,3095,2011-07-10,2011-07-13,A,1533,Colon Cancer,Malignant neoplasm of sigmoid colon,Mal neo sigmoid colon,3 days


In [278]:
# Which treatment plan [drug(s)] do you think would be indicated to be used as first line of treatment for breast cancer?
"""
Initial approach was to look at number of treatments with each drug,
and duration of treatment with each drug.
Then, I noticed that some patients start off on a drug and then end up using another drug.
To find best first line of treatment, I need to fine drug that people start with and stop after finishing treatment.
"""
# Treatment counts with each drug
bc = tdc[tdc.Diagnosis == 'Breast Cancer']
breast_cancer = bc.groupby('Drug_Code')
breast_cancer = breast_cancer['Patient_ID'].nunique().to_frame('Treatment_Count').reset_index()
print(breast_cancer)

# Drug treatment duration for breast cancer
treatment_duration = bc.sort_values('Treatment_Date').groupby(['Patient_ID','Diagnosis_Date'], as_index=False).last()
treatment_duration['Diagnosis_Date'] = pd.to_datetime(treatment_duration['Diagnosis_Date'])
treatment_duration['Treatment_Date'] = pd.to_datetime(treatment_duration['Treatment_Date'])
treatment_duration['Treatment_Duration'] = (treatment_duration['Treatment_Date'] - treatment_duration['Diagnosis_Date']).dt.days
mean_treatment_duration = treatment_duration.groupby('Drug_Code', as_index=False)['Treatment_Duration'].mean() # Average duration of treatment in days
print(mean_treatment_duration)
# Find combinations of drug code and patients
drug_patient_combinations = bc.groupby(['Patient_ID', 'Drug_Code']).size().reset_index()
print(drug_patient_combinations)

  Drug_Code  Treatment_Count
0         A               19
1         B               13
2         C               15
  Drug_Code  Treatment_Duration
0         A                  50
1         B                 550
2         C                  81
    Patient_ID Drug_Code   0
0         2038         A   8
1         2038         B   8
2         2120         A  13
3         2120         B  13
4         2407         A   9
5         2407         B   9
6         2425         A  17
7         2425         B  17
8         2462         A  18
9         2462         B  18
10        2735         A  12
11        2735         B  12
12        2763         A  15
13        2763         B  15
14        3757         C  18
15        3948         A  16
16        3948         B  16
17        3948         C   9
18        4256         C  30
19        4354         C  30
20        4374         C  30
21        4692         A  15
22        4692         B  15
23        4692         C  20
24        5259         A  16
25

In [ ]:
"""
In my opinion, the best first line of treatment would be Drug C by itself.
There is a general trend of treatment plans ending with C. 
Patients who start with Drug C, end with Drug C.
Patients who start with other treatments typically end with Drug C.

The other notable treatment plan is to start with Drug A, 
and then continue to Drug B, and then to Drug C if there is no success.

Also looking at the duration of usage with each drugs, it is notable that the
average duration for Drug B is extensively larger than that of Drug A and C.
This is not preferable if Drug B is chemotherapy, due to severe unwanted side effects.
However, if Drug B is immunotherapy, it is less taxing than chemotherapy on the body,
and could potentially be why there is a longer use of it.

Perhaps, Drug A is an anti-cancer drug that demonstrates the least amount of side effects,
which is why doctors prefer to start on Drug A. Then if Drug A doesn't work, they proceed to Drug B,
which is also minimally invasive on standard of life. If Drug B doesn't work, 
doctors will turn to a more intense drug, Drug C, in hopes of finally defeating the cancer.
"""

In [253]:
# How do these drugs compare in terms of duration of therapy?
"""
Take approach similar to finding days from diagnosis to treatment, however now we're looking for last date
of treatment.
"""
treatment_duration = tdc.sort_values('Treatment_Date').groupby(['Patient_ID','Diagnosis_Date'], as_index=False).last()
treatment_duration['Diagnosis_Date'] = pd.to_datetime(treatment_duration['Diagnosis_Date'])
treatment_duration['Treatment_Date'] = pd.to_datetime(treatment_duration['Treatment_Date'])
treatment_duration['Treatment_Duration'] = (treatment_duration['Treatment_Date'] - treatment_duration['Diagnosis_Date']).dt.days
treatment_duration.groupby('Drug_Code', as_index=False)['Treatment_Duration'].mean() # Average duration of treatment in days

,Drug_Code,Treatment_Duration
0,A,50.400000
1,B,363.500000
2,C,95.238095
